# 다익스트라

In [4]:
import heapq

def dijkstra(graph, start):
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    priority_queue = [(0, start)]

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_distance > distances[current_node]:
            continue

        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight

            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances

# Example graph
graph = {
    'A': {'B': 5, 'C': 2},
    'B': {'A': 5, 'C': 1, 'D': 3},
    'C': {'A': 2, 'B': 1, 'D': 8, 'E': 6},
    'D': {'B': 3, 'C': 8, 'E': 4},
    'E': {'C': 6, 'D': 4}
}

start_node = 'A'
shortest_distances = dijkstra(graph, start_node)

print("Shortest distances from node", start_node)
for node, distance in shortest_distances.items():
    print("To node", node, "-> Distance:", distance)
shortest_distances

Shortest distances from node A
To node A -> Distance: 0
To node B -> Distance: 3
To node C -> Distance: 2
To node D -> Distance: 6
To node E -> Distance: 8


{'A': 0, 'B': 3, 'C': 2, 'D': 6, 'E': 8}

# 벨만포드

In [5]:
def bellman_ford(graph, start):
    distances = {node: float('inf') for node in graph}
    distances[start] = 0

    # Relax edges repeatedly
    for _ in range(len(graph) - 1):
        for node in graph:
            for neighbor, weight in graph[node].items():
                if distances[node] + weight < distances[neighbor]:
                    distances[neighbor] = distances[node] + weight

    # Check for negative cycles
    for node in graph:
        for neighbor, weight in graph[node].items():
            if distances[node] + weight < distances[neighbor]:
                # A negative cycle exists
                raise ValueError("Graph contains a negative cycle")

    return distances

# Example graph with negative weights
graph = {
    'A': {'B': 5, 'C': 2},
    'B': {'C': -3, 'D': 3},
    'C': {'D': 8, 'E': -6},
    'D': {'E': 4},
    'E': {}
}

start_node = 'A'
try:
    shortest_distances = bellman_ford(graph, start_node)
    print("Shortest distances from node", start_node)
    for node, distance in shortest_distances.items():
        print("To node", node, "-> Distance:", distance)
except ValueError as e:
    print(e)


Shortest distances from node A
To node A -> Distance: 0
To node B -> Distance: 5
To node C -> Distance: 2
To node D -> Distance: 8
To node E -> Distance: -4


# Disjoint set

In [6]:
class DisjointSet:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])  # Path compression
        return self.parent[x]

    def union(self, x, y):
        root_x, root_y = self.find(x), self.find(y)
        if root_x == root_y:
            return  # They are already in the same set

        # Union by rank to maintain balanced trees
        if self.rank[root_x] < self.rank[root_y]:
            self.parent[root_x] = root_y
        elif self.rank[root_x] > self.rank[root_y]:
            self.parent[root_y] = root_x
        else:
            self.parent[root_x] = root_y
            self.rank[root_y] += 1

# Example usage:
n = 5  # Number of elements
ds = DisjointSet(n)

ds.union(0, 1)
ds.union(2, 3)
ds.union(0, 2)

print(ds.find(1))  # Output: 3 (All elements 0, 1, 2, and 3 belong to the same set)
print(ds.find(4))  # Output: 4 (Element 4 is in its own set)


3
4


# 크루스칼

In [8]:
class DisjointSet:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])  # Path compression
        return self.parent[x]

    def union(self, x, y):
        root_x, root_y = self.find(x), self.find(y)
        if root_x == root_y:
            return  # They are already in the same set

        # Union by rank to maintain balanced trees
        if self.rank[root_x] < self.rank[root_y]:
            self.parent[root_x] = root_y
        elif self.rank[root_x] > self.rank[root_y]:
            self.parent[root_y] = root_x
        else:
            self.parent[root_x] = root_y
            self.rank[root_y] += 1

def kruskal(graph):
    nodes = list(graph.keys())
    node_to_index = {node: i for i, node in enumerate(nodes)}
    edges = []
    for u in graph:
        for v, weight in graph[u].items():
            edges.append((node_to_index[u], node_to_index[v], weight))

    edges.sort(key=lambda x: x[2])  # Sort edges by weight in ascending order
    num_vertices = len(nodes)
    mst = []
    ds = DisjointSet(num_vertices)

    for u, v, weight in edges:
        if ds.find(u) != ds.find(v):
            ds.union(u, v)
            mst.append((nodes[u], nodes[v], weight))

    return mst

# Example graph
graph = {
    'A': {'B': 5, 'C': 2},
    'B': {'A': 5, 'C': 1, 'D': 3},
    'C': {'A': 2, 'B': 1, 'D': 8, 'E': 6},
    'D': {'B': 3, 'C': 8, 'E': 4},
    'E': {'C': 6, 'D': 4}
}

minimum_spanning_tree = kruskal(graph)
print("Minimum Spanning Tree:")
for u, v, weight in minimum_spanning_tree:
    print(f"Edge: {u} - {v}, Weight: {weight}")


Minimum Spanning Tree:
Edge: B - C, Weight: 1
Edge: A - C, Weight: 2
Edge: B - D, Weight: 3
Edge: D - E, Weight: 4


# 위상정렬

In [9]:
from collections import defaultdict, deque

def topological_sort(graph):
    in_degree = defaultdict(int)
    for node in graph:
        for neighbor in graph[node]:
            in_degree[neighbor] += 1

    queue = deque()
    for node in graph:
        if in_degree[node] == 0:
            queue.append(node)

    topological_order = []
    while queue:
        current_node = queue.popleft()
        topological_order.append(current_node)

        for neighbor in graph[current_node]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)

    if len(topological_order) != len(graph):
        return None  # The graph contains a cycle

    return topological_order

# Example graph
graph = {
    'A': ['B', 'C'],
    'B': ['D'],
    'C': ['D', 'E'],
    'D': ['E'],
    'E': []
}

result = topological_sort(graph)
if result is None:
    print("Graph contains a cycle, cannot perform Topological Sort.")
else:
    print("Topological Sort Order:", result)


Topological Sort Order: ['A', 'B', 'C', 'D', 'E']


# 플로이드 워셜 알고리즘

In [10]:
def floyd_warshall(graph):
    num_vertices = len(graph)
    dist = [[float('inf')] * num_vertices for _ in range(num_vertices)]

    # Initialize the distance matrix with known edge weights
    for i in range(num_vertices):
        for j in range(num_vertices):
            if i == j:
                dist[i][j] = 0
            elif j in graph[i]:
                dist[i][j] = graph[i][j]

    # Compute the shortest path distances using dynamic programming
    for k in range(num_vertices):
        for i in range(num_vertices):
            for j in range(num_vertices):
                dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])

    return dist

# Example graph with weighted edges (use 'inf' for non-existent edges)
graph = {
    0: {1: 3, 2: 6},
    1: {2: 2, 3: 1},
    2: {3: 1},
    3: {0: 1}
}

shortest_paths = floyd_warshall(graph)

# Print the shortest path distances between all pairs of vertices
for i, row in enumerate(shortest_paths):
    for j, distance in enumerate(row):
        print(f"Shortest distance from node {i} to node {j}: {distance}")


Shortest distance from node 0 to node 0: 0
Shortest distance from node 0 to node 1: 3
Shortest distance from node 0 to node 2: 5
Shortest distance from node 0 to node 3: 4
Shortest distance from node 1 to node 0: 2
Shortest distance from node 1 to node 1: 0
Shortest distance from node 1 to node 2: 2
Shortest distance from node 1 to node 3: 1
Shortest distance from node 2 to node 0: 2
Shortest distance from node 2 to node 1: 5
Shortest distance from node 2 to node 2: 0
Shortest distance from node 2 to node 3: 1
Shortest distance from node 3 to node 0: 1
Shortest distance from node 3 to node 1: 4
Shortest distance from node 3 to node 2: 6
Shortest distance from node 3 to node 3: 0


# BFS

In [1]:
from collections import deque

def bfs(graph, start):
    visited = set()
    queue = deque([start])

    while queue:
        node = queue.popleft()
        if node not in visited:
            visited.add(node)
            print("Visited node:", node)

            for neighbor in graph[node]:
                if neighbor not in visited:
                    queue.append(neighbor)

# Example graph represented as an adjacency list
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D'],
    'C': ['A', 'E'],
    'D': ['B'],
    'E': ['C', 'F'],
    'F': ['E']
}

start_node = 'A'
bfs(graph, start_node)


Visited node: A
Visited node: B
Visited node: C
Visited node: D
Visited node: E
Visited node: F


# DFS

In [2]:
def dfs(graph, node, visited):
    if node not in visited:
        visited.add(node)
        print("Visited node:", node)

        for neighbor in graph[node]:
            dfs(graph, neighbor, visited)

# Example graph represented as an adjacency list
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D'],
    'C': ['A', 'E'],
    'D': ['B'],
    'E': ['C', 'F'],
    'F': ['E']
}

start_node = 'A'
visited_nodes = set()
dfs(graph, start_node, visited_nodes)


Visited node: A
Visited node: B
Visited node: D
Visited node: C
Visited node: E
Visited node: F
